In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.optimizers import adam_v2
import urllib.request
import ssl
import os

In [2]:
# Define the input image size
img_width, img_height = 224, 224

# Set the number of epochs and batch size
epochs = 20
batch_size = 32

In [3]:
# Set the path to the dataset 
train_dir = '/Volumes/SSD/dishes'
valid_dir = '/Volumes/SSD/dishes-val'
test_dir = '/Volumes/SSD/dishes-test'

In [4]:
# Create the data generators
# Create a data generator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Create a data generator for the validation set
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')

Found 1985 images belonging to 2 classes.
Found 833 images belonging to 2 classes.


In [5]:
# Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
# Download the resnet50 weights manually
weights_path = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
if not os.path.exists(weights_path):
    url = 'https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(url, weights_path)

In [7]:
# Load the ResNet50 model pre-trained on ImageNet and remove the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

2023-04-08 13:42:03.724312: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Add a global average pooling layer and a dense layer with sigmoid activation for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

In [9]:
# Compile the model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=adam_v2.Adam(learning_rate=0.0001, decay=0.0001/epochs), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

Epoch 1/20
63/63 [==============================] - 980s 15s/step - loss: 0.4135 - accuracy: 0.7985 - val_loss: 1.9678 - val_accuracy: 0.4682
Epoch 2/20
63/63 [==============================] - 914s 15s/step - loss: 0.1545 - accuracy: 0.9416 - val_loss: 0.8716 - val_accuracy: 0.4694
Epoch 3/20
63/63 [==============================] - 914s 15s/step - loss: 0.0797 - accuracy: 0.9698 - val_loss: 0.7301 - val_accuracy: 0.4682
Epoch 4/20
63/63 [==============================] - 916s 15s/step - loss: 0.0825 - accuracy: 0.9733 - val_loss: 0.7577 - val_accuracy: 0.5282
Epoch 5/20
63/63 [==============================] - 913s 14s/step - loss: 0.0601 - accuracy: 0.9758 - val_loss: 1.2611 - val_accuracy: 0.5282
Epoch 6/20
63/63 [==============================] - 915s 15s/step - loss: 0.0606 - accuracy: 0.9788 - val_loss: 0.7647 - val_accuracy: 0.5030
Epoch 7/20
63/63 [==============================] - 914s 15s/step - loss: 0.0291 - accuracy: 0.9919 - val_loss: 0.7957 - val_accuracy: 0.5246
Epoch 

In [12]:
# Evaluate the model
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Found 2321 images belonging to 2 classes.
73/73 [==============================] - 232s 3s/step - loss: 904.0524 - accuracy: 0.5536
Test accuracy: 0.5536406636238098
